 ## <center>Model Training ##
 ---

In [4]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
from sklearn.neighbors import KNeighborsRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor,AdaBoostRegressor
from sklearn.svm import SVR
from sklearn.linear_model import LinearRegression,Ridge,Lasso
import warnings
warnings.filterwarnings("ignore")

In [6]:
df = pd.read_csv(r"C:\ML_project1\notebook\data\student.csv")
df.head()

,gender,race_ethnicity,parental_level_of_education,lunch,test_preparation_course,math_score,reading_score,writing_score
0,female,group B,bachelor's degree,standard,none,72,72,74
1,female,group C,some college,standard,completed,69,90,88
2,female,group B,master's degree,standard,none,90,95,93
3,male,group A,associate's degree,free/reduced,none,47,57,44
4,male,group C,some college,standard,none,76,78,75


#### we will predict the math score

In [7]:
X = df.drop(columns=['math_score'])
y = df['math_score']

In [9]:
num_features = X.select_dtypes(include=[np.number]).columns
cat_features = X.select_dtypes(exclude=[np.number]).columns

from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer

numeric_transformer = StandardScaler()
categorical_transformer = OneHotEncoder(handle_unknown='ignore')

preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, num_features),
        ('cat', categorical_transformer, cat_features)
    ])


In [12]:
X_transformed = preprocessor.fit_transform(X)
X_transformed.shape


(1000, 19)

In [14]:
from sklearn.model_selection import train_test_split
X_train , X_test , y_train , y_test = train_test_split(X_transformed, y, test_size=0.2, random_state=42)


In [15]:
def evaluate_model(true,predicted):
    mae = mean_absolute_error(true,predicted)
    print(f"Mean Absolute Error: {mae}")
    mse = mean_squared_error(true,predicted)
    print(f"Mean Squared Error: {mse}")
    r2 = r2_score(true,predicted)
    print(f"R^2 Score: {r2}")


In [18]:
models = {
    'linear_regression': LinearRegression(),
    'decision_tree': DecisionTreeRegressor(),
    'random_forest': RandomForestRegressor(),
    'svm': SVR(),
    'knn': KNeighborsRegressor(),
    'gradient_boosting': GradientBoostingRegressor(),
    'lasso': Lasso()
}
r2_scores = {}
for name, model in models.items():
    model.fit(X_train, y_train)
    preds = model.predict(X_test)
    print(f"Evaluation results for {name}:")
    evaluate_model(y_test, preds)
    r2_scores[name] = r2_score(y_test, preds)
    print("\n\n")

Evaluation results for linear_regression:
Mean Absolute Error: 4.214763142474851
Mean Squared Error: 29.095169866715487
R^2 Score: 0.8804332983749565



Evaluation results for decision_tree:
Mean Absolute Error: 6.3
Mean Squared Error: 63.87
R^2 Score: 0.7375260131569863



Evaluation results for random_forest:
Mean Absolute Error: 4.612025
Mean Squared Error: 35.189087513888886
R^2 Score: 0.8553903226375743



Evaluation results for svm:
Mean Absolute Error: 5.4015392444969965
Mean Squared Error: 66.04200493745648
R^2 Score: 0.7286001513223705



Evaluation results for knn:
Mean Absolute Error: 5.631
Mean Squared Error: 52.661
R^2 Score: 0.7835894375897926



Evaluation results for gradient_boosting:
Mean Absolute Error: 4.307564153267947
Mean Squared Error: 31.091214358431653
R^2 Score: 0.8722305466032831



Evaluation results for lasso:
Mean Absolute Error: 5.157879138921817
Mean Squared Error: 42.50633235127345
R^2 Score: 0.825320079562973





In [20]:
for name, score in r2_scores.items():
    print(f"R^2 score for {name}: {score}")


R^2 score for linear_regression: 0.8804332983749565
R^2 score for decision_tree: 0.7375260131569863
R^2 score for random_forest: 0.8553903226375743
R^2 score for svm: 0.7286001513223705
R^2 score for knn: 0.7835894375897926
R^2 score for gradient_boosting: 0.8722305466032831
R^2 score for lasso: 0.825320079562973


In [26]:
from sklearn.model_selection import GridSearchCV
param_grid = {
    'linear_regression':{
        'fit_intercept': [True, False],
        'copy_X': [True, False]
    },
    'gradient_boosting':{
        'n_estimators': [100, 200],
        'learning_rate': [0.01, 0.1, 0.2]
    }
}
best_models = {}
for name, model in models.items():
    if name in param_grid:
        grid_search = GridSearchCV(model, param_grid[name], cv=5, scoring='r2', n_jobs=-1)
        grid_search.fit(X_train, y_train)
        best_models[name] = grid_search.best_estimator_
        print(f"Best parameters for {name}: {grid_search.best_params_}")



Best parameters for linear_regression: {'copy_X': True, 'fit_intercept': True}
Best parameters for gradient_boosting: {'learning_rate': 0.1, 'n_estimators': 100}


In [29]:
from sklearn.metrics import r2_score, mean_squared_error

for name, model in best_models.items():
    y_pred = model.predict(X_test)
    r2 = r2_score(y_test, y_pred)
    mse = mean_squared_error(y_test, y_pred)
    rmse = mse ** 0.5
    print(f"{name} -> R2: {r2:.4f}, RMSE: {rmse:.4f}")

linear_regression -> R2: 0.8804, RMSE: 5.3940
gradient_boosting -> R2: 0.8720, RMSE: 5.5800


In [30]:
for name, model in best_models.items():
    train_r2 = model.score(X_train, y_train)
    test_r2 = model.score(X_test, y_test)
    print(f"{name} -> Train R²: {train_r2:.4f}, Test R²: {test_r2:.4f}")


linear_regression -> Train R²: 0.8743, Test R²: 0.8804
gradient_boosting -> Train R²: 0.9050, Test R²: 0.8720


### After hyper parameter tuning the best model i can choose is linear regression madel ##